In [ ]:
!conda install -y gdown

/bin/bash: line 1: conda: command not found


In [ ]:
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import cv2
import random
from tqdm import tqdm
import h5py
import gdown
from torchvision import models

import os
import re

import torch
import torch.optim as optim
from torchvision import transforms

from torch.utils.data import Dataset
import os

from sklearn.metrics import f1_score as sklearn_f1
from sklearn.metrics import confusion_matrix
import torchvision
from sklearn.model_selection import StratifiedKFold
from torchvision.transforms import Normalize
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn

import torch.cuda.amp as amp
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torch.nn.functional as F
from torchvision.ops import nms

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

from PIL import Image

from sklearn.model_selection import train_test_split
import timm
from sklearn.metrics import mean_absolute_error
import torch.nn.functional as F

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


SEED = 777
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = "/content/drive/MyDrive/human_settlement_mapping/train_data.h5"
test_path = "/content/drive/MyDrive/human_settlement_mapping/test_data.h5"

In [ ]:
class CFG:
    seed = 20
    N_folds = 2
    train_folds = [0, 1, 2, 3, 4, 5]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    apex = True  # use half precision
    epochs = 1
    weights = torch.tensor([0.206119, 0.793881], dtype=torch.float32)
    clip_val = 1000.
    batch_size =20
    gradient_accumulation_steps = 4
    lr = 1e-4
    weight_decay = 1e-2
    num_workers=os.cpu_count()
    mixed_precision = True

In [ ]:
class BalancedSatelliteImageDataset(Dataset):
    def __init__(self, hdf5_file, indices=None, transform=None, oversample_factor=1):
        self.transform = transform
        self.oversample_factor = oversample_factor
        with h5py.File(hdf5_file, 'r') as hdf:
            if indices is not None:
                self.images = np.array(hdf['images'])[indices]
                self.labels = np.array(hdf['labels'])[indices]
            else:
                self.images = np.array(hdf['images'])
                self.labels = np.array(hdf['labels'])

        self.ones_indices = np.where(self.labels == 1)[0]
        self.zeros_indices = np.where(self.labels == 0)[0]

        if len(self.ones_indices) < len(self.zeros_indices):
            self.minority_indices = self.ones_indices
            self.majority_indices = self.zeros_indices
        else:
            self.minority_indices = self.zeros_indices
            self.majority_indices = self.ones_indices
        self.oversampled_minority_indices = np.repeat(self.minority_indices, self.oversample_factor)
        self.balanced_indices = np.concatenate([self.majority_indices, self.oversampled_minority_indices])
        np.random.shuffle(self.balanced_indices)

        self.resize = transforms.Resize((224, 224))

    def __len__(self):
        return len(self.balanced_indices)

    def __getitem__(self, idx):
        true_idx = self.balanced_indices[idx]
        image = self.images[true_idx]
        label = self.labels[true_idx]
        image = np.mean(image, axis=0)  # Average over the channels to reduce to 1 channel
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
        image = self.resize(image)  # Resize the image to 224x224
        if self.transform:
            image = self.transform(image)
        image = image.permute(0, 2, 1)  # Convert to (channels, height, width) if necessary
        return image, torch.tensor(label, dtype=torch.long)

In [ ]:
class CustomVisionTransformer(nn.Module):
    def __init__(self, num_classes):
        super(CustomVisionTransformer, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, in_chans=1, num_classes=num_classes)

    def forward(self, x):
        return self.vit(x)




class CustomSwinTransformer(nn.Module):
    def __init__(self, num_classes):
        super(CustomSwinTransformer, self).__init__()
        # Load Swin Transformer and modify to accept 16 channels
        self.swin_transformer = timm.create_model('swin_base_patch4_window7_224', pretrained=True, in_chans=1, num_classes=num_classes)

    def forward(self, x):
        return self.swin_transformer(x)


In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def train_epoch(cfg, model, train_loader, criterion, optimizer, scheduler, epoch):
    model.train()
    loss_hist = Averager()
    scaler = torch.amp.GradScaler('cuda', enabled=cfg.mixed_precision)
    correct = 0
    total = 0
    steps = 0

    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    optimizer.zero_grad()

    for batch_idx, (inputs, targets) in pbar:
        inputs, targets = inputs.to(cfg.device), targets.to(cfg.device)

        with torch.amp.autocast('cuda', enabled=cfg.mixed_precision):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss / cfg.gradient_accumulation_steps).backward()
        if (batch_idx + 1) % cfg.gradient_accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
            scaler.unscale_(optimizer)  # Unscale gradients before clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=cfg.clip_val)

            scaler.step(optimizer)
            scaler.update()

            if scheduler is not None:
                scheduler.step()  # Move scheduler.step() here

            optimizer.zero_grad()

        loss_hist.send(loss.item())

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        steps += 1
        pbar.set_postfix({'Loss': loss_hist.value, 'Acc': 100.*correct/total})

    return loss_hist.value, 100.*correct/total


def validate(cfg, model, valid_loader, criterion):
    model.eval()
    loss_hist = Averager()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valid_loader):
            inputs, targets = inputs.to(cfg.device), targets.to(cfg.device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss_hist.send(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return loss_hist.value, 100.*correct/total


In [ ]:
def main():

    torch.manual_seed(CFG.seed)
    np.random.seed(CFG.seed)

    with h5py.File(train_path, 'r') as hdf:
        all_labels = np.array(hdf['labels'])
        num_samples = len(hdf['images'])

    num_classes = np.unique(all_labels).shape[0]

    skf = StratifiedKFold(n_splits=CFG.N_folds, shuffle=True, random_state=CFG.seed)

    for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(num_samples), all_labels)):
        print(f"Fold {fold + 1}")

        train_dataset = BalancedSatelliteImageDataset(train_path, indices=train_idx)
        valid_dataset = BalancedSatelliteImageDataset(train_path, indices=val_idx)

        train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers, pin_memory=True)

        model = CustomVisionTransformer(num_classes)
        model = model.to(CFG.device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.epochs)

        best_val_acc = 0
        for epoch in range(CFG.epochs):
            train_loss, train_acc = train_epoch(CFG, model, train_loader, criterion, optimizer, scheduler, epoch)
            val_loss, val_acc = validate(CFG, model, valid_loader, criterion)

            print(f"Epoch {epoch + 1}/{CFG.epochs} Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}% Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}%")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f'best_model_fold_{fold}.pth')
                print(f"Saved best model with validation accuracy: {best_val_acc:.2f}%")

        print(f"Best validation accuracy for fold {fold + 1}: {best_val_acc:.2f}%")

if __name__ == "__main__":
    main()

Fold 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1:   0%|          | 3/27500 [00:02<4:16:56,  1.78it/s, Loss=1.45, Acc=11.7]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoch 1: 100%|██████████| 27500/27500 [1:16:13<00:00,  6.01it/s, Loss=0.296, Acc=90.9]


In [ ]:
5